In [ ]:
import pandas as pd
import numpy as np
import os
import sklearn
import random 
from scipy.stats import mode

In [ ]:
import pymysql
import mysql.connector

In [ ]:
# Pyspark - SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit

In [ ]:
# Pyspark - ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
#Make Spark Session
spark = SparkSession.builder.config( "spark.jars","/root/hadoop/share/hadoop/tools/lib/mysql-connector-java-8.0.27.jar") \
    .master("local").appName("b2c").getOrCreate()

In [ ]:
#Load tables (buylist, product, category_small, user_recommand)
buylist = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "buylist") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
product = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "product") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
category_small = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "category_small") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
user_recommand = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "user_recommand") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()

In [ ]:
buylist.filter(col('user_id')==1).show()

In [ ]:
user_recommand.toPandas()

In [ ]:
user_id = 1

In [ ]:
count = buylist.filter(col('user_id')==user_id).count()

In [ ]:
products_id = buylist.filter(col('user_id')==user_id).toPandas().product_id

In [ ]:
cate_s_list = [0 for _ in range(count)]
cate_m_list = [0 for _ in range(count)]

In [ ]:
for i in range(count):
    cate_s_list[i] = product.where(col('id') == int(products_id[i])).toPandas().category_small_id
    
for i in range(count):
    cate_m_list[i] = category_small.where(col('id') == int(cate_s_list[i])).toPandas().category_mid_id

In [ ]:
mode(cate_m_list).mode

In [ ]:
conn = pymysql.connect(host='b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com',
                       user='admin',password='SMoEcEXZ6PZUUiqDv5w9', 
                       db='b2c', charset='utf8')

sql = "INSERT INTO 'user_recommand' VALUES (%s,%s)"

with conn:
    with conn.cursor() as cur:
        cur.execute(sql, (user_id, int(mode(cate_m_list).mode)))       
        conn.commit()